In [1]:
import pandas as pd

In [2]:
flight_data_df = pd.read_csv("flight_data_files/jan_2019_ontime.csv")

In [3]:
print(f"Number of rows in the 2019 data file is: {len(flight_data_df)}")

Number of rows in the 2019 data file is: 583985


In [4]:
flight_data_df_list = [flight_data_df]

In [5]:
flight_data_df = pd.read_csv("flight_data_files/jan_2020_ontime.csv")

In [6]:
print(f"Number of rows in the 2020 data file is: {len(flight_data_df)}")

Number of rows in the 2020 data file is: 607346


In [7]:
flight_data_df_list.append(flight_data_df)

In [8]:
flight_data_df = pd.concat(flight_data_df_list, axis=0, ignore_index=True)

In [9]:
print(f"Number of rows in the combined data file is: {len(flight_data_df)}")

Number of rows in the combined data file is: 1191331


In [10]:
flight_data_df.nunique()

DAY_OF_MONTH               31
DAY_OF_WEEK                 7
OP_UNIQUE_CARRIER          17
OP_CARRIER_AIRLINE_ID      17
OP_CARRIER                 17
TAIL_NUM                 5857
OP_CARRIER_FL_NUM        6997
ORIGIN_AIRPORT_ID         353
ORIGIN_AIRPORT_SEQ_ID     373
ORIGIN                    353
DEST_AIRPORT_ID           353
DEST_AIRPORT_SEQ_ID       373
DEST                      353
DEP_TIME                 1440
DEP_DEL15                   2
DEP_TIME_BLK               19
ARR_TIME                 1440
ARR_DEL15                   2
CANCELLED                   2
DIVERTED                    2
DISTANCE                 1512
Unnamed: 21                 0
dtype: int64

In [11]:
airport_ORIGIN_code_list = flight_data_df["ORIGIN"].unique().tolist()

In [12]:
print(f"Number of ORIGIN airport codes in the data file is: {len(airport_ORIGIN_code_list)}")

Number of ORIGIN airport codes in the data file is: 353


In [13]:
airport_DEST_code_list = flight_data_df["DEST"].unique().tolist()

In [14]:
print(f"Number of DEST airport codes in the data file is: {len(airport_DEST_code_list)}")

Number of DEST airport codes in the data file is: 353


In [15]:
airport_code_list = airport_ORIGIN_code_list 
    
for airport_code in airport_DEST_code_list:
    if airport_code not in airport_code_list:
        airport_code_list.append(airport_code)

In [16]:
print(f"Number of unique airport codes in the data file is: {len(airport_code_list)}")

Number of unique airport codes in the data file is: 353


In [17]:
import io

airport_code_city_dict = {}
airport_codes_found_cnt = 0

with io.open("flight_data_files/airport_codes_city_raw.txt", 'rt', newline='\r\n') as f:
    line = f.readline()[0:-2]
    while line != 'Canadian Provinces':
        
        if len(line.strip()) > 1 and line not in ['return to top']:
            
            line_split    = line.split('(')
            city          = line_split[0].rstrip()
            airport_code  = line_split[1][0:3]
            
            if airport_code in airport_code_list:
                
                if airport_code in airport_code_city_dict:
                    if city.find("/") == -1:
                        current_city = airport_code_city_dict[airport_code]
                        comma_index  = current_city.index(",")
                        airport_code_city_dict[airport_code] = current_city[:comma_index] + "/" + city + current_city[comma_index:]
                else:
                    airport_code_city_dict[airport_code] = city
                
                airport_codes_found_cnt += 1
            
        line = f.readline()[0:-2]

In [18]:
print(f"The number of airport codes found is: {airport_codes_found_cnt}")

The number of airport codes found is: 363


In [19]:
for airport_code in airport_code_city_dict:
    city = airport_code_city_dict[airport_code]
    if city.find("/") > -1:
        print(f"Airport code: {airport_code} cities {city}")

Airport code: CAK cities Akron/Canton, OH
Airport code: ABE cities Allentown/Bethlehem/Easton, PA, PA, PA
Airport code: AVL cities Asheville/Hendersonville, NC, NC
Airport code: HTS cities Ashland, KY/Huntington, WV
Airport code: MBS cities Bay City/Midland/Saginaw, MI, MI, MI
Airport code: BPT cities Beaumont/Port Arthur, TX
Airport code: GPT cities Biloxi/Gulfport/Gulfport, MS, MS
Airport code: BGM cities Binghamton/Endicott/Johnson City, NY, NY, NY
Airport code: TRI cities Bristol/Johnson City/Kingsport, TN, TN, VA
Airport code: MRY cities Carmel/Monterey, CA, CA
Airport code: CMI cities Champaign/Urbana, IL
Airport code: HIB cities Chisholm/Hibbing, MN
Airport code: PIE cities Clearwater/St Petersburg, FL
Airport code: COD cities Cody/Yellowstone, WY
Airport code: DFW cities Dallas/Fort Worth, TX
Airport code: DTW cities Detroit, MI - Metro/Wayne County
Airport code: VPS cities Fort Walton Beach/Valparaiso, FL, FL
Airport code: GGG cities Gladewater/Kilgore/Longview, TX, TX
Airport

In [20]:
for airport_code in airport_code_city_dict:
    
    city = airport_code_city_dict[airport_code]
    
    if city.find("/") > -1:
        city_split = city.split(",")
        split_length = len(city_split)
        city = city_split[0] + "," + city_split[1]
        
        if (split_length > 2 and city_split[1] != city_split[2]):
            city = city + "," + city_split[2]
        
        if (split_length > 3 and city_split[2] != city_split[3]):
            city = city + "," + city_split[3]
            
        if (split_length > 4 and city_split[3] != city_split[4]):
            city = city + "," + city_split[3]
            
        if (split_length > 5 and city_split[4] != city_split[4]):
            city = city + "," + city_split[4]
           
        airport_code_city_dict[airport_code] = city
        
        print(f"Airport code: {airport_code} cities {city}")

Airport code: CAK cities Akron/Canton, OH
Airport code: ABE cities Allentown/Bethlehem/Easton, PA
Airport code: AVL cities Asheville/Hendersonville, NC
Airport code: HTS cities Ashland, KY/Huntington, WV
Airport code: MBS cities Bay City/Midland/Saginaw, MI
Airport code: BPT cities Beaumont/Port Arthur, TX
Airport code: GPT cities Biloxi/Gulfport/Gulfport, MS
Airport code: BGM cities Binghamton/Endicott/Johnson City, NY
Airport code: TRI cities Bristol/Johnson City/Kingsport, TN, VA
Airport code: MRY cities Carmel/Monterey, CA
Airport code: CMI cities Champaign/Urbana, IL
Airport code: HIB cities Chisholm/Hibbing, MN
Airport code: PIE cities Clearwater/St Petersburg, FL
Airport code: COD cities Cody/Yellowstone, WY
Airport code: DFW cities Dallas/Fort Worth, TX
Airport code: DTW cities Detroit, MI - Metro/Wayne County
Airport code: VPS cities Fort Walton Beach/Valparaiso, FL
Airport code: GGG cities Gladewater/Kilgore/Longview, TX
Airport code: GSO cities Greensboro/High Point, NC
Airp

In [21]:
airport_code_city_dict["DTW"] = "Detroit, MI"
airport_code_city_dict["HOU"] = "Hobby-Houston, TX"
airport_code_city_dict["OGG"] = "Kahului - Maui, HI"
airport_code_city_dict["LIH"] = "Lihue - Kauai, HI"
airport_code_city_dict["TTN"] = "Trenton/Mercer NJ"
airport_code_city_dict["SEA"] = "Seattle/Tacoma, WA"
airport_code_city_dict["TRI"] = "Bristol, TN/Johnson City/Kingsport, VA"

In [22]:
airport_city_code_df = pd.DataFrame.from_dict(airport_code_city_dict, orient="index", columns=["City"])

In [23]:
airport_city_code_df.head(400)

,City
ABR,"Aberdeen, SD"
ABI,"Abilene, TX"
ADK,"Adak Island, AK"
CAK,"Akron/Canton, OH"
ALB,"Albany, NY"
...,...
ILM,"Wilmington, NC"
ORH,"Worcester, MA"
WRG,"Wrangell, AK"
YAK,"Yakutat, AK"


In [24]:
airport_city_code_df.reset_index(inplace=True)
airport_city_code_df.rename(columns={"index":"Airport Code"}, inplace=True)

In [25]:
airport_city_code_df.head()

,Airport Code,City
0,ABR,"Aberdeen, SD"
1,ABI,"Abilene, TX"
2,ADK,"Adak Island, AK"
3,CAK,"Akron/Canton, OH"
4,ALB,"Albany, NY"


In [26]:
airport_city_code_df.to_csv("flight_data_files/airport_codes_city.csv", index=False)